In [1]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
from scipy import signal
import os
import matplotlib.pyplot as plt

# ================= STEP 1: SETUP =================
base_path = r"D:\impress_project\eeg_signals\data\LRMI-21679035\organized_data_v2"
output_dir = os.path.join(base_path, "raw_data_v2")
os.makedirs(output_dir, exist_ok=True)

# Load events
events_path = os.path.join(base_path, "events", "task-motor-imagery_events.tsv")
events_df = pd.read_csv(events_path, sep='\t')

# Get MI events (value=2, 4000ms duration)
mi_events = events_df[(events_df['value'] == 2) & (events_df['duration'] == 4000)].copy()

# ================= STEP 2: LOAD RAW .MAT FILE =================
subject_id = "01"
mat_path = os.path.join(base_path, "raw_data", f"sub-{subject_id}_task-motor-imagery_eeg.mat")

print(f"Loading raw .mat file: {mat_path}")
data = loadmat(mat_path)
eeg_struct = data['eeg'][0, 0]

# Extract data - NOTE: This needs verification based on actual structure
rawdata_full = eeg_struct['rawdata'][0]  # Shape might be (33, total_samples)

print(f"Raw data shape: {rawdata_full.shape}")

# The key issue: We need to know if this is 1 trial or all 40 trials concatenated
# From your README: 40 trials × 4000 samples = 160,000 samples
expected_total_samples = 40 * 4000

if rawdata_full.shape[1] == expected_total_samples:
    print(f"✓ Data has {expected_total_samples} samples (40 trials concatenated)")
    # Reshape to (40 trials, 33 channels, 4000 samples)
    n_trials = 40
    n_channels = 33
    n_samples = 4000
    
    # Reshape: (33, 160000) → (33, 40, 4000) → (40, 33, 4000)
    rawdata = rawdata_full.reshape(n_channels, n_trials, n_samples)
    rawdata = np.transpose(rawdata, (1, 0, 2))
    
elif rawdata_full.shape[1] == 4000:
    print(f"⚠️ Only 4000 samples - might be just 1 trial")
    print("Assuming this is trial 0, need to check other files")
    rawdata = rawdata_full.reshape(1, 33, 4000)  # Single trial
else:
    print(f"⚠️ Unexpected shape: {rawdata_full.shape[1]} samples")
    rawdata = None

if rawdata is not None:
    print(f"Reshaped data: {rawdata.shape}")
    print(f"Expected: (40, 33, 4000)")

Loading raw .mat file: D:\impress_project\eeg_signals\data\LRMI-21679035\organized_data_v2\raw_data\sub-01_task-motor-imagery_eeg.mat
Raw data shape: (33, 4000)
⚠️ Only 4000 samples - might be just 1 trial
Assuming this is trial 0, need to check other files
Reshaped data: (1, 33, 4000)
Expected: (40, 33, 4000)


In [2]:
# ================= STEP 3: VERIFY STRUCTURE =================
def check_mat_structure(subject_num):
    """Check the structure of a .mat file"""
    mat_path = os.path.join(base_path, "raw_data", f"sub-{subject_num:02d}_task-motor-imagery_eeg.mat")
    
    if not os.path.exists(mat_path):
        print(f"File not found: {mat_path}")
        return None
    
    data = loadmat(mat_path)
    eeg_struct = data['eeg'][0, 0]
    rawdata = eeg_struct['rawdata'][0]
    label = eeg_struct['label'][0]
    
    return {
        'shape': rawdata.shape,
        'label': label,
        'channels': rawdata.shape[0],
        'samples': rawdata.shape[1]
    }

# Check first 3 subjects
print("\nChecking .mat file structures:")
for sub_num in range(1, 4):
    info = check_mat_structure(sub_num)
    if info:
        print(f"Subject {sub_num:02d}: shape={info['shape']}, label={info['label']}")


Checking .mat file structures:
Subject 01: shape=(33, 4000), label=[1]
Subject 02: shape=(33, 4000), label=[1]
Subject 03: shape=(33, 4000), label=[1]


In [3]:
# ================= STEP 4: ERD/ERS CALCULATION =================
def calculate_erd_ers_for_subject(rawdata, labels, fs=500):
    """
    Calculate ERD/ERS as per paper methodology
    rawdata: shape (n_trials, n_channels, n_samples)
    labels: array of labels (1=left, 2=right)
    """
    
    # Paper's parameters
    alpha_band = (8, 13)  # Hz
    beta_band = (13, 30)  # Hz
    motor_bands = (8, 30)  # Combined for ERD/ERS
    
    # C3 = channel 12 (0-based), C4 = channel 13 (0-based)
    c3_idx = 12
    c4_idx = 13
    
    # Apply 8-30 Hz bandpass filter (2nd order Butterworth, zero-phase)
    nyquist = fs / 2
    low = 8 / nyquist
    high = 30 / nyquist
    b, a = signal.butter(2, [low, high], btype='band')
    
    # Filter C3 and C4 data
    c3_filtered = np.zeros_like(rawdata[:, c3_idx, :])
    c4_filtered = np.zeros_like(rawdata[:, c4_idx, :])
    
    for trial in range(rawdata.shape[0]):
        c3_filtered[trial] = signal.filtfilt(b, a, rawdata[trial, c3_idx, :])
        c4_filtered[trial] = signal.filtfilt(b, a, rawdata[trial, c4_idx, :])
    
    # Calculate power (amplitude^2)
    c3_power = c3_filtered ** 2
    c4_power = c4_filtered ** 2
    
    # Separate by condition
    left_idx = np.where(labels == 1)[0]  # Left hand MI
    right_idx = np.where(labels == 2)[0]  # Right hand MI
    
    # Average power for each condition
    c3_power_left = c3_power[left_idx].mean(axis=0)
    c4_power_left = c4_power[left_idx].mean(axis=0)
    c3_power_right = c3_power[right_idx].mean(axis=0)
    c4_power_right = c4_power[right_idx].mean(axis=0)
    
    # Baseline: first 500 samples (0-1 second)
    # Paper says baseline is -1 to 0 seconds, but our data starts at 0
    # Using first 500 samples as baseline (consistent with your visualization)
    baseline_start, baseline_end = 0, 500
    
    def calculate_erd_ers_percentage(power_signal):
        M = power_signal[baseline_start:baseline_end].mean()
        return ((power_signal - M) / M) * 100
    
    # Calculate ERD/ERS
    erd_ers = {
        'c3_left': calculate_erd_ers_percentage(c3_power_left),
        'c4_left': calculate_erd_ers_percentage(c4_power_left),
        'c3_right': calculate_erd_ers_percentage(c3_power_right),
        'c4_right': calculate_erd_ers_percentage(c4_power_right)
    }
    
    return erd_ers, c3_filtered, c4_filtered

# Get labels from events for subject 01
# Filter events - we need to know how to map events to subjects
# Events file doesn't have subject column, so assuming it's sequential
subject_events = mi_events.iloc[:40]  # First 40 events for subject 01
labels = subject_events['trial_type'].values  # trial_type: 1=left?, 2=right?

print(f"\nLabels from events: {labels}")
print(f"Left hand: {np.sum(labels == 1)}, Right hand: {np.sum(labels == 2)}")


Labels from events: [1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1
 2 1 2]
Left hand: 20, Right hand: 20


In [4]:
# ================= STEP 5: VISUALIZE ERD/ERS =================
def plot_erd_ers(erd_ers, subject_id, fs=500):
    """Plot ERD/ERS curves like Figure 4 in paper"""
    
    time = np.arange(4000) / fs  # 0 to 4 seconds
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    fig.suptitle(f'Subject {subject_id} - ERD/ERS Analysis (8-30 Hz)', fontsize=14)
    
    # Smoothing (as in paper: smoothness coefficient = 200)
    def smooth(data, window=200):
        return np.convolve(data, np.ones(window)/window, mode='same')
    
    # Plot 1: Left Hand MI
    ax1 = axes[0, 0]
    ax1.plot(time, smooth(erd_ers['c3_left']), 'b-', label='C3', linewidth=2)
    ax1.plot(time, smooth(erd_ers['c4_left']), 'r-', label='C4', linewidth=2)
    ax1.axhline(y=0, color='k', linestyle='--', alpha=0.3)
    ax1.axvline(x=1.0, color='g', linestyle='--', alpha=0.5, label='Baseline end')
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('ERD/ERS (%)')
    ax1.set_title('Left Hand MI')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Right Hand MI
    ax2 = axes[0, 1]
    ax2.plot(time, smooth(erd_ers['c3_right']), 'b-', label='C3', linewidth=2)
    ax2.plot(time, smooth(erd_ers['c4_right']), 'r-', label='C4', linewidth=2)
    ax2.axhline(y=0, color='k', linestyle='--', alpha=0.3)
    ax2.axvline(x=1.0, color='g', linestyle='--', alpha=0.5, label='Baseline end')
    ax2.set_xlabel('Time (s)')
    ax2.set_ylabel('ERD/ERS (%)')
    ax2.set_title('Right Hand MI')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Plot 3: C3 Comparison
    ax3 = axes[1, 0]
    ax3.plot(time, smooth(erd_ers['c3_left']), 'b-', label='Left MI', linewidth=2)
    ax3.plot(time, smooth(erd_ers['c3_right']), 'r-', label='Right MI', linewidth=2)
    ax3.axhline(y=0, color='k', linestyle='--', alpha=0.3)
    ax3.set_xlabel('Time (s)')
    ax3.set_ylabel('ERD/ERS (%)')
    ax3.set_title('C3: Left vs Right MI')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Plot 4: C4 Comparison
    ax4 = axes[1, 1]
    ax4.plot(time, smooth(erd_ers['c4_left']), 'b-', label='Left MI', linewidth=2)
    ax4.plot(time, smooth(erd_ers['c4_right']), 'r-', label='Right MI', linewidth=2)
    ax4.axhline(y=0, color='k', linestyle='--', alpha=0.3)
    ax4.set_xlabel('Time (s)')
    ax4.set_ylabel('ERD/ERS (%)')
    ax4.set_title('C4: Left vs Right MI')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print(f"\nERD/ERS Summary for Subject {subject_id}:")
    for condition in ['left', 'right']:
        c3_mean = erd_ers[f'c3_{condition}'].mean()
        c4_mean = erd_ers[f'c4_{condition}'].mean()
        print(f"{condition.capitalize()} Hand MI:")
        print(f"  C3: Mean ERD/ERS = {c3_mean:.2f}%")
        print(f"  C4: Mean ERD/ERS = {c4_mean:.2f}%")

# Calculate and plot
if rawdata is not None and len(rawdata) == len(labels):
    erd_ers_results, c3_filtered, c4_filtered = calculate_erd_ers_for_subject(rawdata, labels)
    plot_erd_ers(erd_ers_results, subject_id)
else:
    print(f"\n⚠️ Data shape mismatch: rawdata has {rawdata.shape[0]} trials, labels has {len(labels)}")


⚠️ Data shape mismatch: rawdata has 1 trials, labels has 40


In [5]:
# ================= STEP 6: SAVE LABELED RAW DATA =================
def save_labeled_raw_data(subject_id, rawdata, labels, output_dir):
    """Save labeled raw data in numpy format"""
    
    # Create subject directory
    subject_dir = os.path.join(output_dir, subject_id)
    os.makedirs(subject_dir, exist_ok=True)
    
    # Save data
    data_path = os.path.join(subject_dir, f"{subject_id}_raw_labeled.npz")
    
    # Save with metadata
    np.savez_compressed(
        data_path,
        raw_data=rawdata,  # (n_trials, 33, 4000)
        labels=labels,     # (n_trials,)
        sampling_rate=500,
        trial_duration=4.0,
        n_trials=len(rawdata),
        n_channels=33,
        n_samples_per_trial=4000
    )
    
    print(f"✓ Saved labeled raw data for {subject_id}")
    print(f"  Location: {data_path}")
    print(f"  Shape: {rawdata.shape}")
    print(f"  Labels: {np.unique(labels, return_counts=True)}")
    
    return data_path

if rawdata is not None and len(rawdata) == len(labels):
    save_labeled_raw_data(f"sub-{subject_id}", rawdata, labels, output_dir)

In [6]:
# Check what's actually in the .mat file for labels
mat_path = os.path.join(base_path, "raw_data", "sub-01_task-motor-imagery_eeg.mat")
data = loadmat(mat_path)
eeg_struct = data['eeg'][0, 0]

print("Examining the 'label' field in .mat file:")
print(f"Type of label field: {type(eeg_struct['label'])}")
print(f"Shape of label field: {eeg_struct['label'].shape}")

# Get the actual label data
label_field = eeg_struct['label']
print(f"\nRaw label field:\n{label_field}")

# If it's an object array, access it properly
if label_field.dtype == 'O':
    print("\nIt's an object array - accessing content:")
    actual_labels = label_field[0]
    print(f"Actual labels shape: {actual_labels.shape}")
    print(f"Actual labels: {actual_labels}")
    
    # Check if it's a single value or array
    if actual_labels.shape == (1,):
        print(f"Single label value: {actual_labels[0]}")
    elif len(actual_labels.shape) == 1:
        print(f"1D array of labels: {actual_labels}")
        print(f"Length: {len(actual_labels)}")
        print(f"Unique values: {np.unique(actual_labels)}")
    elif len(actual_labels.shape) == 2:
        print(f"2D array of labels: shape {actual_labels.shape}")
        print(f"Labels: {actual_labels}")
        
elif label_field.dtype in [np.float64, np.int32, np.int64]:
    print(f"\nNumeric array: {label_field}")
    print(f"Flattened: {label_field.flatten()}")
    print(f"Shape: {label_field.shape}")
    
# Also check the rawdata shape properly
print("\n" + "="*50)
print("Checking rawdata structure:")
rawdata_field = eeg_struct['rawdata']

if rawdata_field.dtype == 'O':
    actual_rawdata = rawdata_field[0]
    print(f"Actual rawdata shape: {actual_rawdata.shape}")
    
    # Check dimensions
    if len(actual_rawdata.shape) == 2:
        print(f"2D array: {actual_rawdata.shape}")
        print(f"Could be: {actual_rawdata.shape[0]} channels × {actual_rawdata.shape[1]} samples")
        
        # If shape is (33, X), X could be:
        # - 4000 (1 trial)
        # - 160000 (40 trials concatenated: 40 × 4000)
        
        n_samples = actual_rawdata.shape[1]
        if n_samples == 4000:
            print("4000 samples = 1 trial (8 seconds at 500Hz)")
        elif n_samples == 160000:
            print("160000 samples = 40 trials concatenated (40 × 4000)")
        else:
            print(f"Unexpected number of samples: {n_samples}")
            
    elif len(actual_rawdata.shape) == 3:
        print(f"3D array: {actual_rawdata.shape}")
        print(f"Should be: (40 trials, 33 channels, 4000 samples)")
else:
    print(f"Rawdata type: {rawdata_field.dtype}")
    print(f"Rawdata shape: {rawdata_field.shape}")

Examining the 'label' field in .mat file:
Type of label field: <class 'numpy.ndarray'>
Shape of label field: (40, 1)

Raw label field:
[[1]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [2]]

Checking rawdata structure:
Rawdata type: float64
Rawdata shape: (40, 33, 4000)


In [9]:
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Thu Nov 17 16:20:34 2022',
 '__version__': '1.0',
 '__globals__': [],
 'eeg': array([[(array([[[8.49312645e+03, 8.49241120e+03, 8.48436457e+03, ...,
                  8.17848094e+03, 8.17784952e+03, 8.17747512e+03],
                 [5.22586021e+03, 5.22440734e+03, 5.22161337e+03, ...,
                  5.28323713e+03, 5.28301665e+03, 5.28198544e+03],
                 [1.04677690e+04, 1.04672772e+04, 1.04657126e+04, ...,
                  1.04541344e+04, 1.04538724e+04, 1.04536203e+04],
                 ...,
                 [4.07098580e+04, 4.07150213e+04, 4.07149319e+04, ...,
                  4.06870369e+04, 4.06856207e+04, 4.06823654e+04],
                 [3.60827564e+04, 3.60855280e+04, 3.60847233e+04, ...,
                  3.59955845e+04, 3.59950964e+04, 3.59942881e+04],
                 [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
                  0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],
 


In [11]:
data['eeg'][0, 0]['label']

array([[1],
       [2],
       [1],
       [2],
       [1],
       [2],
       [1],
       [2],
       [1],
       [2],
       [1],
       [2],
       [1],
       [2],
       [1],
       [2],
       [1],
       [2],
       [1],
       [2],
       [1],
       [2],
       [1],
       [2],
       [1],
       [2],
       [1],
       [2],
       [1],
       [2],
       [1],
       [2],
       [1],
       [2],
       [1],
       [2],
       [1],
       [2],
       [1],
       [2]], dtype=uint8)

In [12]:
# Check what's actually in the .mat file for labels
mat_path = os.path.join(base_path, "raw_data", "sub-01_task-motor-imagery_eeg.mat")
data = loadmat(mat_path)
eeg_struct = data['eeg'][0, 0]

print("Examining the 'label' field in .mat file:")
print(f"Type of label field: {type(eeg_struct['label'])}")
print(f"Shape of label field: {eeg_struct['label'].shape}")

# Get the actual label data
label_field = eeg_struct['label']
print(f"\nRaw label field:\n{label_field}")

# If it's an object array, access it properly
if label_field.dtype == 'O':
    print("\nIt's an object array - accessing content:")
    actual_labels = label_field[0]
    print(f"Actual labels shape: {actual_labels.shape}")
    print(f"Actual labels: {actual_labels}")
    
    # Check if it's a single value or array
    if actual_labels.shape == (1,):
        print(f"Single label value: {actual_labels[0]}")
    elif len(actual_labels.shape) == 1:
        print(f"1D array of labels: {actual_labels}")
        print(f"Length: {len(actual_labels)}")
        print(f"Unique values: {np.unique(actual_labels)}")
    elif len(actual_labels.shape) == 2:
        print(f"2D array of labels: shape {actual_labels.shape}")
        print(f"Labels: {actual_labels}")
        
elif label_field.dtype in [np.float64, np.int32, np.int64]:
    print(f"\nNumeric array: {label_field}")
    print(f"Flattened: {label_field.flatten()}")
    print(f"Shape: {label_field.shape}")
    
# Also check the rawdata shape properly
print("\n" + "="*50)
print("Checking rawdata structure:")
rawdata_field = eeg_struct['rawdata']

if rawdata_field.dtype == 'O':
    actual_rawdata = rawdata_field[0]
    print(f"Actual rawdata shape: {actual_rawdata.shape}")
    
    # Check dimensions
    if len(actual_rawdata.shape) == 2:
        print(f"2D array: {actual_rawdata.shape}")
        print(f"Could be: {actual_rawdata.shape[0]} channels × {actual_rawdata.shape[1]} samples")
        
        # If shape is (33, X), X could be:
        # - 4000 (1 trial)
        # - 160000 (40 trials concatenated: 40 × 4000)
        
        n_samples = actual_rawdata.shape[1]
        if n_samples == 4000:
            print("4000 samples = 1 trial (8 seconds at 500Hz)")
        elif n_samples == 160000:
            print("160000 samples = 40 trials concatenated (40 × 4000)")
        else:
            print(f"Unexpected number of samples: {n_samples}")
            
    elif len(actual_rawdata.shape) == 3:
        print(f"3D array: {actual_rawdata.shape}")
        print(f"Should be: (40 trials, 33 channels, 4000 samples)")
else:
    print(f"Rawdata type: {rawdata_field.dtype}")
    print(f"Rawdata shape: {rawdata_field.shape}")

Examining the 'label' field in .mat file:
Type of label field: <class 'numpy.ndarray'>
Shape of label field: (40, 1)

Raw label field:
[[1]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [2]]

Checking rawdata structure:
Rawdata type: float64
Rawdata shape: (40, 33, 4000)
